In [39]:
import pandas as pd
import numpy as np
import datetime
import quandl
import config

# settings to observe x amount of rows in dataframe
pd.set_option('max_rows', 500)

In [40]:
# Get 500 companies
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500.columns = sp500.iloc[0] 
sp500.drop(sp500.index[0], inplace=True)
sp500 = sp500['Ticker symbol'].tolist()

In [41]:
# API Key
my_api_key = config.api_key



# Loop through tickers given
tickers = ['FB', 'GOOGL']

symbols = []

for sym in tickers:
    
    if sym in tickers:
        sym = sym.replace(".", "_")
    
    start = "2016-01-01"
    end = "2016-12-31"
    
    mydata = quandl.get("WIKI/{}.11".format(sym), start_date=start, end_date=end)
    mydata['Symbol'] = sym
    symbols.append(mydata)
    
    
# attach tickers tables
mydata = pd.concat(symbols)

# index 'Date' to column
mydata.reset_index(inplace=True)

# Change location of column 'Symbol'
symb = mydata['Symbol']
mydata.drop(labels=['Symbol'], axis=1,inplace = True)
mydata.insert(1, 'Symbol', symb)
mydata.head(1)

,Date,Symbol,Adj. Close
0,2016-01-04,FB,102.22


In [42]:
# Stock Daily Return
d_returns = mydata.groupby('Symbol').pct_change()
mydata['Daily_Return'] = d_returns
mydata.head(3)

# Mean Daily Return
d_mean_return = mydata.groupby('Symbol').Daily_Return.transform('mean').to_frame()
mydata['Mean_Daily_Return'] = d_mean_return

# Std Daily
d_std_return = mydata.groupby('Symbol').Daily_Return.transform('std').to_frame()
mydata['Std_Daily_Return'] = d_std_return

# Daily Sharpe Ratio
sr = d_mean_return/d_std_return
mydata['Daily_Sharpe_Ratio'] = sr

# Annualize Sharpe Ratio
asr = (252**0.5) * sr
mydata['Annualize_Sharpe_Ratio'] = asr

mydata.head()

,Date,Symbol,Adj. Close,Daily_Return,Mean_Daily_Return,Std_Daily_Return,Daily_Sharpe_Ratio,Annualize_Sharpe_Ratio
0,2016-01-04,FB,102.22,NaN,0.000626,0.01785,0.035056,0.556496
1,2016-01-05,FB,102.73,0.004989,0.000626,0.01785,0.035056,0.556496
2,2016-01-06,FB,102.97,0.002336,0.000626,0.01785,0.035056,0.556496
3,2016-01-07,FB,97.92,-0.049043,0.000626,0.01785,0.035056,0.556496
4,2016-01-08,FB,97.33,-0.006025,0.000626,0.01785,0.035056,0.556496
